# Generating the final matrix 

In [1]:
import pandas as pd
import numpy as np

In [2]:
nyt = pd.read_csv("data/nyt_cleaned.csv", sep = "|")

In [3]:
nyt.drop(columns = ['cleaned'], inplace = True)

In [4]:
bonap = pd.read_csv("data/bonap.csv", sep = "|")

In [5]:
epic = pd.read_csv("data/epicurious.csv", sep = "|")

In [12]:
allrecipes = pd.read_csv("data/allrecipes.csv", sep = "|")

In [13]:
columns = bonap.columns

In [14]:
allrecipes.columns = columns

In [15]:
df = pd.concat([nyt, bonap, epic, allrecipes], axis = 0)

In [16]:
metamasterlist = pd.read_csv("masterlist.csv")

In [17]:
metamasterlist.drop_duplicates(inplace = True)

In [18]:
masterlist = metamasterlist['ingredient'].to_list()

In [19]:
masterlist = sorted(masterlist, key=len, reverse = True)

In [20]:
# Function, copy-pasted from previous cleaning notebook
def simplify(row):

    rowresults = []
    row = row.replace('[\'', '\']')
    row = row.replace(']', '') 
    items = row.split(', ')
    for item in items: 
        for ingredient in masterlist:
            if ingredient in item: 
                rowresults.append(ingredient)
                break
    return list(set(rowresults))

In [21]:
df['cleaned'] = df['ingredients'].apply(simplify)

In [22]:
df.reset_index(drop=True, inplace=True)

In [23]:
df.head()

,name,url,ingredients,cleaned
0,Harissa-Roasted Sweet Potatoes and Red Onion,https://cooking.nytimes.com/recipes/1023541-ha...,"['3 medium sweet potatoes, washed and trimmed,...","[sweet potato, feta, ground cumin, lemon, hari..."
1,Tofu and Mushroom Jorim (Soy-Braised Tofu),https://cooking.nytimes.com/recipes/1023476-to...,"['1/3 cup low-sodium soy sauce', '5 garlic clo...","[oil, soy sauce, brown sugar, kimchi, scallion..."
2,Roasted Chicken With Crispy Mushrooms,https://cooking.nytimes.com/recipes/1023551-ro...,"['2 to 2 1/4 pounds boneless, skinless chicken...","[tarragon, red onion, oyster, shiitake, chicke..."
3,Chocolate Pumpkin Swirl Muffins,https://cooking.nytimes.com/recipes/1023565-ch...,"['2 cups/256 grams all-purpose flour', '1 tabl...","[kosher salt, yogurt, baking powder, vanilla e..."
4,Pasta e Patate (Pasta and Potato Soup),https://cooking.nytimes.com/recipes/1023564-pa...,"['1/3 cup extra-virgin olive oil', '1 large ye...","[pecorino, celery, carrot, spaghetti, rosemary..."


In [24]:
df.shape

(104353, 4)

In [25]:
type(df)

pandas.core.frame.DataFrame

In [26]:
drop = list(df.loc[df.astype(str).duplicated()].index.values)

In [1]:
#drop

In [28]:
df.drop(drop, axis = 0, inplace = True)

In [29]:
df.to_csv('df_final.csv', sep = '|', index = False)

In [30]:
df.shape

(102263, 4)

In [31]:
biglist = []
for i in range(len(df)):
    smallist = []
    for ingredient in masterlist: 
        if ingredient in df['cleaned'].iloc[i]:    
            smallist.append(1)
        else: 
            smallist.append(0)
    biglist.append(smallist)

In [32]:
matrix = pd.DataFrame(data = biglist, columns = masterlist)

In [33]:
matrix.to_csv('matrix.csv', index = False)

In [34]:
droplist = []
for column in matrix.columns: 
    if matrix[column].sum() == 0:
        droplist.append(column)
#    droplist.append([column,matrix[column].sum()])

In [35]:
len(droplist)

0

In [36]:
droplist

[]

In [37]:
metamasterlist = metamasterlist[~metamasterlist['ingredient'].isin(droplist)]

In [38]:
metamasterlist.reset_index(drop=True, inplace=True)

In [39]:
metamasterlist

,ingredient,meat
0,green bean,0.0
1,mincemeat,1.0
2,beef,1.0
3,graham crackers,0.0
4,leek,0.0
...,...,...
806,lard,1.0
807,pear,0.0
808,chicken backs,1.0
809,salami,1.0


In [40]:
metamasterlist.to_csv('masterlist.csv', index = False)